In [1]:
#News or Noise - Algo Trading Signals

In [101]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from dotenv import load_dotenv
import panel as pn
from panel.interact import interact
from panel import widgets
import alpaca_trade_api as tradeapi
import quandl
import os
import requests

In [67]:
# Pull in price data for ETF's from Alpaca API from 1/1/2010 - 12/31/2020
# SPY - S&P 500
# XLF - Financials Sector
# XLK - Technology
# XTL - Telecom Sector
# XLU - Utilities
# XLI - Industrials
# XLV - Healthcare
# XLY - Consumer Discretionary
# XLP - Consumer Staples
# XLB - Materials
# XOP - Oil & Gas
# XHB - Homebuilders
# XME - Metals and Miners
# XRT - Retail


In [3]:
# Pull in crypto currency price data from Krakken API from 1/1/2010 - 12/31/2020
# BTC
# ETH
# LTC

In [102]:
load_dotenv()

True

In [103]:
#import

api_key_QD = os.getenv("QUANDL_API_KEY")
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# type(api_key_QD)
# type(alpaca_api_key)

In [104]:
#import alpaca api file for ETF price history
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
start_date = pd.Timestamp("2010-01-02", tz="America/New_York").isoformat()
today = pd.Timestamp("2020-12-31", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["SPY", "XLF", "XLK", "XTL","XLU","XLI", "XLV", "XLY", "XLP", "XLB", "XOP", "XHB", "XME", "XRT"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get closing prices
df_ETFs = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = today
).df

# Preview DataFrame
# YOUR CODE HERE!
df_ETFs.head()

# Output the data to CSV
df_ETFs.to_csv("Resources/ETFs1.csv", encoding='utf-8', index=True)

In [109]:
ETF_csv = Path("Resources/ETFs1.csv")

In [113]:
ETF_df = pd.read_csv(ETF_csv, index_col='Unnamed: 0', infer_datetime_format=True, parse_dates=True)
ETF_df.head()
ETF_df.dtype

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [109]:
#ETF_df.rename(columns={'Unnamed: 0': 'Date'}, inplace = True)

In [109]:
ETF_df = ETF_df.drop(ETF_df.index[0])
#ETF_df['Date'] = pd.to_datetime(ETF_df['Date'], utc=True)
ETF_df.index = pd.to_datetime(ETF_df.index, utc=True)
ETF_df['Date']= ETF_df['Date'].dt.strftime('%m/%d/%Y')
ETF_df.sort_index(inplace=True)

#drop unneccessary columns and keep close price columns
ETF_close_prices_df = ETF_df.loc[:, ['SPY.3', 'XLF.3', 'XLK.3', 'XTL.3', 'XLU.3', 'XLI.3', 'XLV.3',
                                    'XLY.3', 'XLP', 'XLB.3', 'XOP.3', 'XHB.3', 'XME.3', 'XRT.3']]
ETF_close_prices_df.rename(columns={'SPY.3': 'SPY',
                   'XLF.3': 'XLF',
                   'XLK.3': 'XLK',
                   'XTL.3': 'XTL',
                   'XLU.3': 'XLU',
                   'XLI.3': 'XLI',
                   'XLV.3': 'XLV',
                   'XLY.3': 'XLY',
                   'XLP.3': 'XLP',
                   'XLB.3': 'XLB',
                   'XOP.3': 'XOP',
                   'XHB.3': 'XHB',
                   'XME.3': 'XME',
                   'XRT.3': 'XRT', 
                  }, inplace=True)

ETF_close_prices_df[['SPY', 'XLF', 'XLK', 'XTL', 'XLU', 'XLI', 
                     'XLV', 'XLY', 'XLP', 'XLB', 'XOP', 'XHB', 'XME', 'XRT']] = ETF_close_prices_df[['SPY', 'XLF', 'XLK', 'XTL', 'XLU', 'XLI', 
                     'XLV', 'XLY', 'XLP', 'XLB', 'XOP', 'XHB', 'XME', 'XRT']].astype(float)
ETF_close_prices_df.dtypes

#calculate daily returns
ETF_daily_returns_df = ETF_close_prices_df.pct_change().dropna()
ETF_daily_returns_df.head()

ParserError: Unknown string format: time